<a href="https://colab.research.google.com/github/AkshayaAgarwal/Roadlanedetection/blob/main/roadlanedetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q opencv-python

In [2]:
import pandas as pd
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from moviepy import editor
import moviepy

In [9]:

def region_selection(image):
  mask = np.zeros_like(image)

  if len(image.shape) > 2:
    channel_count = image.shape[2]
    igjnore_mask_color = (255,) * channel_count

  else:
    ignore_mask_color= 255

    rows, cols = image.shape[:2]
    bottom_left = [cols * 0.1, rows * 0.95]
    top_left = [cols * 0.4, rows * 0.6]
    bottom_right = [ cols * 0.9, rows * 0.95]
    top_right = [cols * 0.6, rows * 0.6]
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)

    cv2.fillPoly(mask, vertices, ignore_mask_color)

    masked_image= cv2.bitwise_and(image, mask)
    return masked_image

def hough_transform(image):

  rho=1

  theta = np.pi/180

  threshold = 20

  minLineLength = 20

  maxLineGap = 500

  return cv2.HoughLinesP(image, rho = rho, theta = theta, threshold = threshold,
                         minLineLength = minLineLength, maxLineGap = maxLineGap)



def average_slope_intercept(lines):
    """
    Find the slope and intercept of the left and right lanes of each image.
    Parameters:
        lines: output from Hough Transform
    """
    left_lines    = [] #(slope, intercept)
    left_weights  = [] #(length,)
    right_lines   = [] #(slope, intercept)
    right_weights = [] #(length,)

    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 == x2:
                continue
            # calculating slope of a line
            slope = (y2 - y1) / (x2 - x1)
            # calculating intercept of a line
            intercept = y1 - (slope * x1)
            # calculating length of a line
            length = np.sqrt(((y2 - y1) ** 2) + ((x2 - x1) ** 2))
            # slope of left lane is negative and for right lane slope is positive
            if slope < 0:
                left_lines.append((slope, intercept))
                left_weights.append((length))
            else:
                right_lines.append((slope, intercept))
                right_weights.append((length))
    #
    left_lane  = np.dot(left_weights,  left_lines) / np.sum(left_weights)  if len(left_weights) > 0 else None
    right_lane = np.dot(right_weights, right_lines) / np.sum(right_weights) if len(right_weights) > 0 else None
    return left_lane, right_lane

def pixel_points(y1, y2, line):
    """
    Converts the slope and intercept of each line into pixel points.
        Parameters:
            y1: y-value of the line's starting point.
            y2: y-value of the line's end point.
            line: The slope and intercept of the line.
    """
    if line is None:
        return None
    slope, intercept = line
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    return ((x1, y1), (x2, y2))

def lane_lines(image, lines):
    """
    Create full lenght lines from pixel points.
        Parameters:
            image: The input test image.
            lines: The output lines from Hough Transform.
    """
    left_lane, right_lane = average_slope_intercept(lines)
    y1 = image.shape[0]
    y2 = y1 * 0.6
    left_line  = pixel_points(y1, y2, left_lane)
    right_line = pixel_points(y1, y2, right_lane)
    return left_line, right_line


def draw_lane_lines(image, lines, color=[255, 0, 0], thickness=12):
    """
    Draw lines onto the input image.
        Parameters:
            image: The input test image (video frame in our case).
            lines: The output lines from Hough Transform.
            color (Default = red): Line color.
            thickness (Default = 12): Line thickness.
    """
    line_image = np.zeros_like(image)
    for line in lines:
        if line is not None:
            cv2.line(line_image, *line,  color, thickness)
    return cv2.addWeighted(image, 1.0, line_image, 1.0, 0.0)


def frame_processor(image):

  grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  kernel_size = 5

  blur = cv2.GaussianBlur(grayscale, (kernel_size, kernel_size), 0)
  low_t = 50

  high_t = 150

  edges = cv2.Canny(blur, low_t, high_t)

  region = region_selection(edges)



  hough = hough_transform(region)

  result = draw_lane_lines(image, lane_lines(image, hough))
  return result

def process_video(test_video, output_video):
  input_video=editor.VideoFileClip(test_video, audio=False)

  processed = input_video.fl_image(frame_processor)

  processed.write_videofile(output_video, audio=False)

process_video('test2.mp4','output.mp4')




Moviepy - Building video output.mp4.
Moviepy - Writing video output.mp4



t: 100%|█████████▉| 1296/1297 [01:40<00:00, 14.32it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file test2.mp4, 2764800 bytes wanted but 0 bytes read,at frame 1295/1298, at time 25.90/25.94 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready output.mp4
